<a href="https://colab.research.google.com/github/randiepathirage/4th/blob/main/implementation/Prev_Implementation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data imports
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
sns.barplot(x='frequency',y='trigrams',data=trigrams.head(30))
import matplotlib.pyplot as plt
from tensorflow import keras
from unicodedata import normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import LSTM,Bidirectional,Input,Concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Variable Setup

In [ ]:
model_id = "e5_1"
batch_size = 32 #Batch size for training.
epochs = 50 #Number of epochs to train for.
latent_dim = 256 #Latent dimensionality of the encoding space.
num_samples = 24415 #Number of samples to train on,
test_count = round(num_samples*0.2)

##Load Datasets

In [ ]:
path = "/content/drive/MyDrive/4th year Research/data collection/data/ALL_Unicode.csv"
df = pd.read_csv(path,names=["Singlish","Sinhala"],dtype=str).astype(str)

input_data = pd.DataFrame(df.Singlish.values[:num_samples])
input_data.columns = ['Singlish']

target_data = pd.DataFrame(df.Sinhala.values[:num_samples])
target_data.columns = ['Sinhala']

In [ ]:
input_data

,Singlish
0,\nOw ane
1,Ne ne
2,\t Hari
3,5.30ta nam yanne naa
4,6 shirts kohomath 7000/= ne wenne
...,...
24410,heta nm admission ganna yanna wei neda
24411,"Heta ofc yanawa akila, Man gihin call ekak dennan"
24412,Heta off wena welaw mg ekk dl ynna
24413,Heta office ekata gihin iwara karagannawa


#pre-processing fuctions

In [ ]:
def normalize_text (text):
  text = normalize( 'NFD', text).encode( 'utf-8')
  return text.decode('UTF-8')

In [ ]:
def clean_and_normalize(text):
  text = normalize_text (text.lower())
  return text

In [ ]:
def removeUnnecessaryRepeats(word):
  prev = word[0]
  repeatCount =1
  ans = word[0]
  for i in range(1, len(word)):
    if(word[i]==prev):
      repeatCount+=1
    else:
      repeatCount=1

    if(repeatCount<3):
      ans+=word[i]
      prev=word[i]
  return ans

In [ ]:
def preProcessSinglish(singText):
  vowels = ['a', 'e', 'i', 'o', 'u']
  specialConsonentSuffix = ['h']
  singWords = singText.split()
  finList = []
  for singWord in singWords:
    singWord = removeUnnecessaryRepeats(singWord)
    buf =""
    for i in range(len(singWord)):
      char = singWord[i]

      if(char in vowels):
        if(i<len(singWord)-1):
          if(char == singWord[i+1]):
            buf+=char
          else:
            buf+=char
            finList.append(buf)
            buf=""
        else:
            buf+=char
            finList.append(buf)
            buf=""
      else:
        if(i<len(singWord)-1):
          if(char == singWord[i+1] or (singWord[i+1] not in vowels and singWord[i+1] not in specialConsonentSuffix)):
            buf+=char
            finList.append(buf)
            buf=""
          else:
            buf+=char
        else:
          buf+=char
          finList.append(buf)
          buf=""
    finList.append("$")
  return " ".join(finList)

In [ ]:
def preProcessSinhala(sinText):
  sinWords = sinText.split()
  finList =[]
  for sinWord in sinWords:
    charList = re.findall(r'\w\W?', sinWord.strip())
    finList.extend(charList)
    finList.append("$")
  return "".join(finList)

**bold text**#pre-processing data


In [ ]:
input_data['Singlish'] = input_data['Singlish'].apply(lambda x: preProcessSinglish(clean_and_normalize(x)))
target_data['Sinhala'] = target_data['Sinhala'].apply(lambda x:preProcessSinhala(clean_and_normalize(x))).apply(lambda x: '<S> '+x+' <E>')
#test_input_data['singlish'] = test_input_data['singlish'].apply(lambda x: preProcessSinglish(clean_and_normalize(x)))

*italicized text*#Token Generation 

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
test_input_texts = []
input_TUs = set()
target_TUs = set()

input_texts = input_data.Singlish
target_texts = target_data.Sinhala
#test_input_texts = test_input_data.singlish
#test_target_texts = test_target_data.sinhala

for input_text in input_texts:
  for TU in input_text.split():
    if TU not in input_TUs:
      input_TUs.add(TU)

for target_text in target_texts:
  for TU in target_text.split():
    if TU not in target_TUs:
      target_TUs.add(TU)

for test_input_text in test_input_texts:
  for TU in test_input_text.split():
    if TU not in input_TUs:
      input_TUs.add(TU)

input_TUs = sorted(list(input_TUs))
target_TUs = sorted(list(target_TUs))
num_encoder_tokens = len(input_TUs)
num_decoder_tokens = len(target_TUs)
max_encoder_seq_length = 25
max_decoder_seq_length = 25

input_token_index = dict([(TU,i) for i, TU in enumerate(input_TUs)])
target_token_index = dict([(TU,i) for i, TU in enumerate(target_TUs)])

input_TUs

#Setup Input and Target Data

In [ ]:
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

#test_encoder _input_data = np.zeros(
#(len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

#Encoder decoder data setup

In [ ]:
for i, (input_text, target_text, test_input_text) in enumerate(zip(input_texts, target_texts, test_input_texts)):
  for t, TU in enumerate(input_text.split()):
    encoder_input_data[i, t, input_token_index [TU]] = 1.0
  for t, TU in enumerate(target_text.split()):
    #decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index [TU]] = 1.0
    if t > 0:
      #decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index [TU]] = 1.0

  decoder_input_data[i, t + 1 :, target_token_index ["$"]] = 1.0
  decoder_target_data[i, t:, target_token_index ["$"]] = 1.0

for (i, test_input_text) in enumerate(test_input_texts):
  for t, TU in enumerate(test_input_text.split()):
    test_encoder_input_data[i, t, input_token_index [TU]] = 1.0

NameError: ignored

#Model Setup

In [ ]:
encoder_inputs = keras. Input (shape=(None, num_encoder_tokens))
encoder = keras.layers.Bidirectional(keras.layers.LSTM(latent_dim,return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder (encoder_inputs)
state_h= Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

decoder_inputs = keras. Input (shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM( latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Model Training

In [ ]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_filepath,
  save_weights_only=False,
  monitor='val_loss',
  mode='min',
  save_best_only=True)

early_stop_callback = tf.keras.callbacks.EarlyStopping (patience=5, restore_best_weights=True)

training_limit = num_samples - test_count
#Adam (learning_rate=3e-4)
model.compile(
    optimizer=Adam(learning_rate=3e-4), loss="categorical_crossentropy", metrics=["categorical_accuracy"],
)

model.summary()

history = model.fit(
  [encoder_input_data[:training_limit], decoder_input_data[:training_limit]],
  decoder_target_data[:training_limit],
  batch_size=batch_size,
  epochs=epochs,
  validation_split=0.2,
  callbacks=[model_checkpoint_callback, early_stop_callback]
)
model.save(model_id)

#Inference Model

In [ ]:
model = keras.models.load_model(model_id)

encoder_inputs = model.input[0] # input_1
encoder_outputs, forward_h, forward_c, backward_h, backward_c = model.layers [1].output # lstm_1
state_h_enc = Concatenate()([forward_h, backward_h])
state_c_enc = Concatenate()([forward_c, backward_c])
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1] # input_2
decoder_state_input_h= keras.Input (shape=(latent_dim*2,))
decoder_state_input_c = keras. Input (shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers [5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers [6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model= keras.Model(
[decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

#Decoding

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

reverse_index_df = pd.DataFrame (reverse_target_char_index.items())
reverse_index_df.to_csv (r' reverse_index_df.csv', index = False, header=False)

input_token_index_df = pd.DataFrame.from_dict(input_token_index.items())
input_token_index_df.to_csv (r'input_token_index.csv', index = False, header=False)

target_token_index_df = pd.DataFrame.from_dict(target_token_index.items())
target_token_index_df.to_csv (r'target_token_index.csv', index = False, header=False)

def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)

  target_seq= np.zeros((1, 1, num_decoder_tokens))

  target_seq[0, 0, target_token_index ["<S>"]] = 1.0


  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict( [target_seq] + states_value)
  
    sampled_token_index = np.argmax (output_tokens [0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]

    if sampled_char == "<E>" or len(decoded_sentence) > max_decoder_seq_length:
      stop_condition = True
    else:
      decoded_sentence += sampled_char
      decoded_sentence += ""

    target_seq= np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.0

    states_value = [h, c]
  return decoded_sentence